In [ ]:
pip install pandas scikit-learn openml pymfe tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.0/156.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.3 MB/s eta 0:00:00
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=0befd38fe722a304338298f2b89778e1c35867b7c2c6f297f77ec8b881a29be9
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built liac-arff


In [ ]:
from pathlib import Path
import openml
import pandas as pd
from pymfe.mfe import MFE
from tqdm import tqdm
import warnings

# Suppress precision and invalid value warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


# Meta-feature extraction settings
groups = ["landmarking", "general", "statistical", "model-based", "info-theory", "relative"]
summary_funcs = ["mean", "sd", "count", "histogram", "iq_range", "kurtosis", "max", "median", "min", "quantiles", "range", "skewness"]
scoring = "balanced-accuracy"

def featurize_dataset(task_id):
    # Load data using OpenML task ID
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)

    # Convert to numpy arrays
    X = X.to_numpy()
    y = y.to_numpy()

    # Identify categorical columns
    categorical_cols = list(dataset.get_features_by_type('nominal', [dataset.default_target_attribute]))

    # Check if the dataset is classification
    if dataset.qualities["NumberOfClasses"] <= 1:
        print("Unsupported target type. Skipping.")
        return None

    print(f"Processing dataset: {dataset.name}")
    metafeats = []

    # Extract metafeatures
    mfe = MFE(groups=groups, summary=summary_funcs, random_state=0, score=scoring)
    mfe.fit(X, y, cat_cols=categorical_cols, transform_num=False, transform_cat=None)
    ft = mfe.extract()

    # Consolidate results
    fold_metafeats = {"dataset_name": dataset.name}
    for group in groups:
        ft_group = mfe.parse_by_group(group, ft)
        fold_metafeats.update(
            {f"f__pymfe.{group}.{name}": value for name, value in zip(*ft_group)}
        )
    metafeats.append(fold_metafeats)

    return metafeats

def featurize_all_datasets(task_ids):
    output_file = Path("metafeatures.csv")
    if output_file.exists():
        computed_features = pd.read_csv(output_file)
        computed_features.set_index("dataset_name", inplace=True)
    else:
        computed_features = None

    for task_id in task_ids:
        dataset_name = openml.tasks.get_task(task_id).get_dataset().name
        if computed_features is not None and dataset_name in computed_features.index:
            continue

        print(f"Featurizing task ID: {task_id}")
        dataset_metafeatures = featurize_dataset(task_id)
        if dataset_metafeatures is None:
            continue

        dataset_metafeatures = pd.DataFrame(dataset_metafeatures)
        dataset_metafeatures.set_index("dataset_name", inplace=True)

        if computed_features is None:
            computed_features = dataset_metafeatures
            computed_features = computed_features[sorted(computed_features.columns)]
        else:
            computed_features = pd.concat([dataset_metafeatures, computed_features])

        print("Writing. Do not interrupt...")
        computed_features.to_csv(output_file)

# Specify OpenML task IDs
task_ids = [14965, 9977, 34539, 146606, 7592, 146195, 167119, 167120, 168331, 168330, 168335, 146212,
            168868, 31, 10101, 3913, 3917, 9957, 9946, 3918,
            3903, 37, 9971, 9952, 3902, 49, 43, 9978, 10093, 219, 9976, 6, 53, 11, 15, 16, 14, 32, 3549,
            12, 9981, 18, 28, 2074, 29, 45, 125922, 9960, 9964, 22, 2079, 14969, 3560, 14952, 125920, 23,
            3904, 3022, 9985, 9910, 14970, 3021, 3481, 146824, 146820, 146822, 146195, 146800, 146817,
            146819, 146821, 14954, 167141, 167140, 167125]
featurize_all_datasets(task_ids)


Featurizing task ID: 14965
Processing dataset: bank-marketing


/usr/local/lib/python3.10/dist-packages/pymfe/_internal.py:1281: UserWarning:  * Something went wrong while precomputing 'precompute_can_cors'. Will ignore this method. Error message:
TypeError("OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'").
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


Writing. Do not interrupt...
Featurizing task ID: 9977
Processing dataset: nomao


/usr/local/lib/python3.10/dist-packages/pymfe/_internal.py:1281: UserWarning:  * Something went wrong while precomputing 'precompute_can_cors'. Will ignore this method. Error message:
TypeError("OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'").
  warnings.warn(


Writing. Do not interrupt...
Featurizing task ID: 34539
Processing dataset: Amazon_employee_access
Writing. Do not interrupt...
Featurizing task ID: 146606
Processing dataset: higgs


...

Writing. Do not interrupt...
Featurizing task ID: 167141
Processing dataset: churn


/usr/local/lib/python3.10/dist-packages/pymfe/_internal.py:1281: UserWarning:  * Something went wrong while precomputing 'precompute_can_cors'. Will ignore this method. Error message:
TypeError("OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'").
  warnings.warn(


Writing. Do not interrupt...
Featurizing task ID: 167140
Processing dataset: dna
Writing. Do not interrupt...
Featurizing task ID: 167125
Processing dataset: Internet-Advertisements


/usr/local/lib/python3.10/dist-packages/pymfe/_internal.py:1281: UserWarning:  * Something went wrong while precomputing 'precompute_can_cors'. Will ignore this method. Error message:
TypeError("OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'").
  warnings.warn(


Writing. Do not interrupt...


In [ ]:
pip freeze > requirements.txt